In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import os
from model import model  # Import your model class from model.py

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Load trained models
models_ensemble = []
for file in os.listdir("./models/global_epoch0"):
    if file.endswith('.pt'):
        model_instance = model()  # Instantiate the model class
        model_instance.load_state_dict(torch.load(os.path.join("./models/global_epoch0", file)))
        model_instance.eval()
        models_ensemble.append(model_instance)

# Ensemble prediction



Files already downloaded and verified


In [23]:
print(len(models_ensemble))

10


In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import os
from model import model  # Import your model class from model.py

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load test data
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
def voting_ensemble_predict(models, data_loader):
    predictions = []
    labels = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad(), tqdm(total=len(data_loader)) as progress_bar:
        for inputs, target in data_loader:
            inputs, target = inputs.to(device), target.to(device)
            ensemble_outputs = [F.softmax(model(inputs), dim=1) for model in models]
            # print(ensemble_outputs)
            if ensemble_outputs:
                avg_output = torch.mean(torch.stack(ensemble_outputs), dim=0)
                _, predicted = torch.max(avg_output, 1)
                predictions.extend(predicted.cpu().numpy())
                labels.extend(target.cpu().numpy())
                # print("Predicted:", predicted)  # Print predicted values
            progress_bar.update(1)
                  
    return labels, predictions


# Get labels and predictions using voting ensemble
labels, ensemble_predictions = voting_ensemble_predict(models_ensemble, testloader)

# Calculate accuracy
accuracy = accuracy_score(labels, ensemble_predictions)
print("Ensemble accuracy:", accuracy)

100%|██████████| 157/157 [00:27<00:00,  5.67it/s]

Ensemble accuracy: 0.6304


In [15]:
network = models_ensemble[0]
total = 0
correct = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = network(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

IndexError: list index out of range

Testing code based on the json file

In [1]:
client_clusters = {
    "3": [
        "client_0",
        "client_1",
        "client_9"
    ],
    "5": [
        "client_2"
    ],
    "1": [
        "client_3"
    ],
    "2": [
        "client_4"
    ],
    "4": [
        "client_5",
        "client_6"
    ],
    "6": [
        "client_7"
    ],
    "7": [
        "client_8"
    ]
}

In [5]:
from model import model1, model2, model3
import torch
tempModel = model1()

In [6]:
models_ensemble = []
for key, members in client_clusters.items():
    average_model = {}

    for ke, val in  tempModel.state_dict().items():
         average_model[ke] = torch.zeros_like(val)
    
    for name in members:
        model_state_dict = torch.load("./models/global_epoch1/" + name + ".pt")
        for key in average_model:
            average_model[key] += model_state_dict[key] / len(members)
    models_ensemble.append(average_model)

FileNotFoundError: [Errno 2] No such file or directory: './models/global_epoch1/client_0.pt'